In [1]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import networkx as nx

from collections import defaultdict

import scipy.sparse as spsparse
import matplotlib.pylab as plt


import pyscisci.all as pyscisci



In [2]:
path2aps = '/home/ajgates/APS'
path2aps = '/Volumes/GatesExpansionDrive/DataSets/APS/APS2019'

myaps = pyscisci.APS(path2aps, keep_in_memory=False) 

#myaps.download_from_source(files_to_download='paa_supplement')

In [3]:
pub_df = myaps.pub_df

# limit the publications to those published on/before 1966 
pub_df = pub_df.loc[pub_df['Year'] <= 1966]

# get their references
pub2ref_df = myaps.load_references(filter_dict={'CitingPublicationId':np.sort(pub_df['PublicationId'].unique())})

# and get their authors
pub2author_df = myaps.load_publicationauthoraffiliation(columns = ['PublicationId', 'AuthorId', 'FullName'],
    filter_dict={'PublicationId':np.sort(pub_df['PublicationId'].unique())})
  
aid2name = {aid:name for aid, name in pub2author_df[['AuthorId', 'FullName']].values}
del pub2author_df['FullName']

In [4]:
sc, author2int = pyscisci.diffusion_of_scientific_credit(pub2ref_df, pub2author_df, 
                                        pub_df=pub_df, alpha = 0.9, max_iter = 100, tol = 1.0e-10)

int2aid = {i:aid for aid, i in author2int.items()}

print(sc.shape)

NameError: name 'groupby_total' is not defined

In [ ]:
# print the top k authors
topk = 10

topk_authors = np.argpartition(sc, -topk)[-topk:]
topk_authors = topk_authors[np.argsort(sc[topk_authors])][::-1]

for int_id in topk_authors:
    print(aid2name[int2aid[int_id]], sc[int_id])
